# Breaking Down the Example File to See What It Does

In [3]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams, FreqDist

For the main file, I am working off of another file for a large portion of it. However it is currently not weighting the n-grams properly. I want to go through the original file and see how they do it, but the actual code is super hard for me to read... I want to like hack it to pieces here.

In [4]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


First the init:

need `build_dictionary()` first

In [5]:
def build_dictionary(dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

In [6]:
tries_remaining = 6
guessed_letters = []

full_dictionary_location = "words_250000_train.txt"
test_dictionary_location = "words_test.txt"
full_dictionary = build_dictionary(full_dictionary_location) 
test_dictionary = build_dictionary(test_dictionary_location)
full_dictionary_common_letter_sorted = collections.Counter("".join(full_dictionary)).most_common()
current_dictionary = []
wins = []
lose_words = []

# building relevant data structures such as substring and individual letter frequencies
f = open("words_250000_train.txt", "r")
df = []
for x in f:
    df.append(x[:-1])

temp = []
_2gram = []
_3gram = []
_4gram = []    
_5gram = []

for word in df:
    temp.append(list(word))         
    _2gram.extend(list(ngrams(word, 2, pad_left=True, pad_right=True)))
    _3gram.extend(list(ngrams(word, 3, pad_left=True, pad_right=True)))
    _4gram.extend(list(ngrams(word, 4, pad_left=True, pad_right=True)))
    _5gram.extend(list(ngrams(word, 5, pad_left=True, pad_right=True)))

freq_2 = FreqDist(_2gram)
freq_3 = FreqDist(_3gram)
freq_4 = FreqDist(_4gram)
freq_5 = FreqDist(_5gram)

freq_2 = [(elem, freq_2.get(elem)) for elem in freq_2]
freq_3 = [(elem, freq_3.get(elem)) for elem in freq_3]
freq_4 = [(elem, freq_4.get(elem)) for elem in freq_4]
freq_5 = [(elem, freq_5.get(elem)) for elem in freq_5]

prev_guess = []

vowels = ['a','e','i','o','u']

word_len_dict = {}
for i in range(3, 30):
    word_len_dict[i] = []
    for words in df:
        if(len(words)>i):
            for j in range(len(words)-i+1):
                word_len_dict[i].append(words[j:j+i]) 

In my code, I think I have all of this... except I don't do the thing where I analyze vowel count.

In [7]:
print(full_dictionary_common_letter_sorted[0:10])

[('e', 233745), ('i', 184746), ('a', 179837), ('n', 152259), ('o', 150052), ('r', 149228), ('s', 148462), ('t', 137277), ('l', 122431), ('c', 89367)]


In [8]:
word_len_dict[28]

['cyclotrimethylenetrinitramin',
 'yclotrimethylenetrinitramine',
 'trinitrophenylmethylnitramin',
 'rinitrophenylmethylnitramine']

`word_len_dict` is all of the words arranged into a dictionary by word length... I guess we will see why this is important later

I have all of the `freq_n` in my code

In [9]:
freq_2[0:10]

[(('s', None), 41635),
 (('e', 'r'), 39200),
 (('i', 'n'), 37762),
 (('e', None), 33665),
 (('t', 'i'), 27596),
 (('o', 'n'), 27389),
 (('e', 's'), 27128),
 (('t', 'e'), 26219),
 (('a', 'n'), 25117),
 (('r', 'e'), 24800)]

In my code, I got rid of the ones with `None` because I think I tried to match stuff a bit differently...

That could be an issue?

The main issue (I think) is how the program is actually choosing how to pick something.

In [14]:
def candsort(cands, invalids, vowels, vowel_ratio):
    
    # chooses the highest scoring (most common) and valid candidate letter from candidate letters
    for cand in cands:
        if cand[0] == None or cand[0] in invalids:
            continue
        if cand[0] in vowels and vowel_ratio > 0.5:
            continue
        return cand 
    return ('!', 0, 1)

We have to pass here a LIST of CANDS, which are just the elements inside freq_n stuff, I am pretty sure

This is the main part that matches the n-grams to the individual words.

In [15]:
def ngram(word, index, invalids, freqs, vowel_ratio):
    
    # for each '_' found in the missing word, find valid substring matches and get appropriate weights,
    # giving priority to more complete and longer substrings, tiebreakers between longer substrings are
    # broken by higher weights or frequencies
    
    # candidate tuple structure is: (suggestion, weight, rank) 
    
    freq_2, freq_3, freq_4, freq_5 = freqs
    score1 = ('!', 0, 1)
    score2 = ('!', 0, 1)
    score3 = ('!', 0, 1)
    
    # i. case    
    if index == 0:  
        print("i. case")     
        if word[index+1] == '.':
            print("Got .. exit")
            return ('!', 0, 1)

        # iXXX Case
        if (len(word) >= 4) and ('.' not in word[index+1:index+4]):
                print("iXXX Case")
                cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == None) and 
                                                                        (elem[0][2] == word[index+1]) and 
                                                                        (elem[0][3] == word[index+2]) and 
                                                                        (elem[0][4] == word[index+3])]
                return candsort(cands, invalids, vowels, vowel_ratio)

        # iXX Case
        if (len(word) >= 3) and ('.' not in word[index+1:index+3]):
                print("iXX Case")
                cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == None) and 
                                                                        (elem[0][2] == word[index+1]) and 
                                                                        (elem[0][3] == word[index+2])]
                return candsort(cands, invalids, vowels, vowel_ratio)

        # iX case
        print("iX Case is the only possibility remaining")
        cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == None) and 
                                                                (elem[0][2] == word[index+1])]       
        return candsort(cands, invalids, vowels, vowel_ratio)


    # .i case    
    if index == len(word)-1:
        print(".i Case")      
        if word[index-1] == '.':
            print("Got .. exit")
            return ('!', 0, 1)

        # XXXi case:
        if (len(word) >= 4) and ('.' not in word[index-3:index]):
                print("XXXi Case")
                cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                        (elem[0][1] == word[index-2]) and 
                                                                        (elem[0][2] == word[index-1]) and 
                                                                        (elem[0][4] == None)]
                return candsort(cands, invalids, vowels, vowel_ratio)         

        # XXi case
        
        if (len(word) >= 3) and ('.' not in word[index-2:index]):
                print("XXi Case")
                cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                        (elem[0][1] == word[index-1]) and 
                                                                        (elem[0][3] == None)]
                return candsort(cands, invalids, vowels, vowel_ratio)    

        # Xi case
        print("Xi Case only one remaining")
        cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-1]) and 
                                                                (elem[0][2] == None)]
        return candsort(cands, invalids, vowels, vowel_ratio)


    else:  
        # .i. case
        print(".i. Case")
        if word[index-1] == '.' and word[index+1] == '.':
            print("Got ... so exit")
            return ('!', 0, 1)


        # .iX family
        if word[index-1] == '.'and word[index+1] != '.': 
            print(".iX Group")


            # .iXXXX case
            
            if (len(word) - index >= 5) and (index >= 1) and ('.' not in word[index+1:index+5]):
                    print(".iXXXX Case")
                    cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2]) and 
                                                                            (elem[0][4] == word[index+3])]
                    score1 = candsort(cands, invalids, vowels, vowel_ratio)

            # X.iXX case
            if (len(word) - index >= 3) and (index >= 2) and (word[index+2] != '.') and (word[index-2] != '.'):
                    print("X.iXX Case")
                    cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][3] == word[index+1]) and 
                                                                            (elem[0][4] == word[index+2])]
                    score2 = candsort(cands, invalids, vowels, vowel_ratio)

            # XX.iX case                
            if (len(word) - index >= 2) and (index >= 3) and ('.' not in word[index-3:index-1]):
                    print("XX.iX Case")
                    cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                            (elem[0][1] == word[index-2]) and 
                                                                            (elem[0][4] == word[index+1])]
                    score3 = candsort(cands, invalids, vowels, vowel_ratio)

        
            if (score1 != score2) or (score2 != score3) or (score1 != score3):
                print("Comparing Scores")
                best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]                    


            # .iXX case            
            if (len(word) - index >= 3) and (index >= 1) and (word[index+2] != '.'):
                    print(".iXX case")
                    cands = [(elem[0][0], elem[1], 3) for elem in freq_3 if (elem[0][1] == word[index+1]) and 
                                                                            (elem[0][2] == word[index+2])]
                    score1 = candsort(cands, invalids, vowels, vowel_ratio)

            # X.iX case
            if (len(word) - index >= 2) and (index >= 2) and (word[index-2] != '.'):
                    print("X.iX case")
                    cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][3] == word[index+1])]
                    score2 = candsort(cands, invalids, vowels, vowel_ratio)

            if score1 != score2:
                print("comparing scores")
                best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]        

            # .iX        
            cands = [(elem[0][0], elem[1], 2) for elem in freq_2 if elem[0][1] == word[index+1]]
            print(".iX Case")
            return candsort(cands, invalids, vowels, vowel_ratio)


        # Xi. family
        if word[index-1] != '.'and word[index+1] == '.':
            print("Xi. Family")

            # XXXXi. case
            if (len(word) - index >= 2) and (index >= 4) and ('.' not in word[index-4:index-1]):
                    print("XXXXi case")
                    cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and
                                                                            (elem[0][1] == word[index-2]) and 
                                                                            (elem[0][2] == word[index-1])]
                    score1 = candsort(cands, invalids, vowels, vowel_ratio)

            # XXi.X case                    
            if (len(word) - index >= 3) and (index >= 2) and (word[index+2] != '.') and (word[index-2] != '.'):
                    print("XXi.X Case")
                    cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1]) and 
                                                                            (elem[0][4] == word[index+2])]
                    score2 = candsort(cands, invalids, vowels, vowel_ratio)
            
            # Xi.XX case
            if (len(word) - index >= 4) and (index >= 1) and ('.' not in word[index+2:index+4]):
                    print("Xi.XX Case")
                    cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+2]) and 
                                                                            (elem[0][4] == word[index+3])]
                    score3 = candsort(cands, invalids, vowels, vowel_ratio)

            if (score1 != score2) or (score2 != score3) or (score1 != score3):
                print("Comparing Scores")
                best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]  


            # XXi. case
            if (index >= 2) and (word[index-2] != '.'):
                    print("XXi. Case")
                    cands = [(elem[0][2], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1])]
                    score1 = candsort(cands, invalids, vowels, vowel_ratio)
            
            # Xi.X case
            if (len(word) - index >= 3) and (index >= 1) and (word[index+2] != '.'):
                    print("Xi.X Case")
                    cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+2])]
                    score2 = candsort(cands, invalids, vowels, vowel_ratio)

            if score1 != score2:
                print("Comparing Scores")
                best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]     

            # Xi. case
            print("Xi. Case")
            cands = [(elem[0][1], elem[1], 2) for elem in freq_2 if elem[0][0] == word[index-1]]
            return candsort(cands, invalids, vowels, vowel_ratio)


        # XiX family
        if word[index-1] != '.'and word[index+1] != '.':
            print("XiX Case")
            
            # XXiXX case
            if (len(word) - index >= 3) and (index >= 2) and ('.' not in word[index-2:index+3]):
                    cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+1]) and
                                                                            (elem[0][4] == word[index+2])]
                    print("XXiXX Case")
                    score1 = candsort(cands, invalids, vowels, vowel_ratio)

            # XiXXX case
            if (len(word) - index >= 4) and (index >= 1) and ('.' not in word[index-1:index+4]):
                    cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2]) and
                                                                            (elem[0][4] == word[index+3])]
                    print("XiXXX Case")
                    score2 = candsort(cands, invalids, vowels, vowel_ratio)

            # XXXiX case
            if (len(word) - index >= 2) and (index >= 3) and ('.' not in word[index-3:index+2]):
                    cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                            (elem[0][1] == word[index-2]) and 
                                                                            (elem[0][2] == word[index-1]) and
                                                                            (elem[0][4] == word[index+1])]
                    print("XXXiX Case")
                    score3 = candsort(cands, invalids, vowels, vowel_ratio)

            if (score1 != score2) or (score2 != score3) or (score1 != score3):
                print("Comparing Scores")
                best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]                          


            # XiXX case
            if len(word) - index >= 3 and word[index+2] != '.':
                    print("XiXX Case")
                    cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2])]
                    score1 = candsort(cands, invalids, vowels, vowel_ratio)

            # XXiX case
            if index >= 2 and word[index-2] != '.':
                    print("XXiX Case")
                    cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+1])]
                    score2 = candsort(cands, invalids, vowels, vowel_ratio)

            if score1 != score2:
                print("Comparing Scores")
                best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]            


            # XiX case
            print("XiX Case")
            cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-1]) and 
                                                                    (elem[0][2] == word[index+1])]
            return candsort(cands, invalids, vowels, vowel_ratio)

I don't think these are AS important?

In [16]:
def weighted_count(new_dict):
    # count how many times a letter appears in the dictionary
    dictx = collections.Counter()
    for words in new_dict:
        temp = collections.Counter(words)
        for i in temp:
            temp[i] = 1
            dictx += temp
    return dictx
    
def valid_word(word_dict, guess_word):
    # prunes dictionary if training set word does not match length or characters of word-in-progress
    res = []
    for word in word_dict[len(guess_word)]:
        if re.match(guess_word, word):
            res.append(word)
    return res

def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

Ok, so as usual, I will try to guess the word `apple`, and maybe try passing in `.pp.e` with some set of guesses or something. I think in this code, there isn't a `guess_list` like I have it, I think the `freq`s are pruned based on what the guesses are? There is a `failed_letters` attribute though

So the algorithm should pick things up when there is ONE `.`, and it matches based on the longest length?
I think it should pick something that matches `pp.e` right? Case should be `XXiX`?

But... it should loop over each index right...

In [18]:
correct_word = "apple"
word = ".pp.e"
guess_list = ["s", "p", "t", "e"]
failed_letters = ["s", "t"]

# update the ngrams to remove all failed letters   
freq_2 = [(elem[0], elem[1]) for elem in freq_2 if set(elem[0]).isdisjoint(set(failed_letters))]
freq_3 = [(elem[0], elem[1]) for elem in freq_3 if set(elem[0]).isdisjoint(set(failed_letters))]
freq_4 = [(elem[0], elem[1]) for elem in freq_4 if set(elem[0]).isdisjoint(set(failed_letters))]
freq_5 = [(elem[0], elem[1]) for elem in freq_5 if set(elem[0]).isdisjoint(set(failed_letters))]
freqs = [freq_2, freq_3, freq_4, freq_5]

In [19]:
# get vowel ratio
count = 0
for i in word:
    if i in vowels:
        count += 1
vowel_ratio = count/len(word)    

guess_letter = "!"
clean_word = word

#ngram(word, index, invalids, freqs, vowel_ratio)
# ngram weight guesser
if guess_letter == '!':
    options = []
    for i in range(len(clean_word)):
        if clean_word[i] == '.':
            option = ngram(clean_word, i, guessed_letters, freqs, vowel_ratio)
            print(option)
            options.append(option)

    best_guesses = sorted(options, key = lambda x: (x[2], x[1]), reverse=True)
    #best_guess = best_guesses[0][0]


i. case
iXX Case
('a', 275, 4)
.i. Case
XiX Case
XXiX Case
Comparing Scores
('l', 122, 4)


Ok....... why is a given a length 4? It should match with a length 3 right? Or is it matching with an (a p p NONE) element in freq_4?

OK I GET IT, so the NONEs are used as padding to tell if something IS AT THE BEGINNING OR END OF A WORD!, so in this example, it is 4 because it is being matched with a (NONE a p p) from freq4, SIMILARLY, the ones at the end are matched with a NONE, so if we had `.ppl.', it would find matches of  LENGTH 5 of cases iXXX and XXXi with stuff like (None X X X)

I should be able to do this. My code breaks down the target into n-grams and then compares them to n-grams in the freqs, so I just have to pad it with NONEs at the beginning and end.

However, the n-grams should only start predicting things when there are letters that have actually been guessed.

Also, there is only a match if there is ONE AND ONLY ONE letter to predict. So you can't end up in some situation where an n-gram predicts multiple letters.

The SCORES are just the letter popularity that are contained in the elements of the `freq_` 2nd thing.

So based on this, what do I have to fix?
-Pad the original word with NONE at the beginning and end
-when matching n-grams, they can only differ by ONE letter so that there is only ONE predicted letter
-If there are no matching n-grams, default to some other guessing scheme, like most likely letter or entropy.
-The code also keeps track of the fraction of vowels in the guessed word, and if the guess is a vowel and is above the threshold then it gets cut out.

In [20]:
print(best_guesses)

[('a', 275, 4), ('l', 122, 4)]


In [ ]:
def guess(self, word): # word input example: "_ p p _ e "
        
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################
        
        clean_word = word[::2].replace("_",".")
        len_word = len(clean_word)
        word_len_dict = self.word_len_dict
        vowels = self.vowels
        
        # update dictionary 
        curr_dict = self.current_dictionary
        new_dict = []
        for dict_word in curr_dict:
            if len(dict_word) != len_word:
                continue           
            if re.match(clean_word, dict_word):
                new_dict.append(dict_word)
                
        self.current_dictionary = new_dict
        temp = self.weighted_count(new_dict)
        letter_weights = temp.most_common()

        # filter for failed letters
        failed_letters = []
        for letter in self.guessed_letters:
            if letter not in word:
                failed_letters.append(letter)
        
        # get vowel ratio
        count = 0
        for i in word:
            if i in vowels:
                count += 1
        vowel_ratio = count/len(word)          

        # update the ngrams to remove all failed letters   
        freq_2 = [(elem[0], elem[1]) for elem in self.freq_2 if set(elem[0]).isdisjoint(set(failed_letters))]
        freq_3 = [(elem[0], elem[1]) for elem in self.freq_3 if set(elem[0]).isdisjoint(set(failed_letters))]
        freq_4 = [(elem[0], elem[1]) for elem in self.freq_4 if set(elem[0]).isdisjoint(set(failed_letters))]
        freq_5 = [(elem[0], elem[1]) for elem in self.freq_5 if set(elem[0]).isdisjoint(set(failed_letters))]
        freqs = [freq_2, freq_3, freq_4, freq_5]

        # ---- Guessing ----
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        
        guess_letter = '!'
        for choice, count in letter_weights:
            if choice not in self.guessed_letters:
                if choice in vowels and vowel_ratio > 0.5:
                    self.guessed_letters.append(choice)
                    continue
                guess_letter = choice
                break
        
        # parse through each substring from 3 to half of the length of the target word
        if guess_letter == '!':
            sub_len = round(len_word/2)
            if sub_len >= 3:
                c = collections.Counter()
                for i in range(len_word - sub_len +1):
                    #temp_dict = []
                    #for word in word_len_dict[len_word]:
                    #    if re.match(word, clean_word[i:i+sub_len]):
                    #        temp_dict.append(word)
                    
                    temp_dict = self.valid_word(word_len_dict, clean_word[i:i+sub_len])
                    temp = self.weighted_count(temp_dict)
                    c += temp
                sorted_letter_count = c.most_common()
                
                for letter, _ in sorted_letter_count:
                    if letter not in self.guessed_letters:
                        guess_letter = letter
                        break

        # ngram weight guesser
        if guess_letter == '!':
            options = []
            for i in range(len(clean_word)):
                if clean_word[i] == '.':
                    option = self.ngram(clean_word, i, self.guessed_letters, freqs, vowel_ratio)
                    options.append(option)

            best_guesses = sorted(options, key = lambda x: (x[2], x[1]), reverse=True)
            best_guess = best_guesses[0][0]
            if best_guess != '!':
                return best_guess
        
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            char_weights = self.full_dictionary_common_letter_sorted
            for letter, instance_count in char_weights:
                if letter not in self.guessed_letters:
                    if letter in vowels and vowel_ratio > 0.5:
                        self.guessed_letters.append(letter)
                        continue
                    
                    guess_letter = letter
                    break         
                                       
        return guess_letter